In [1]:
# NYC taxi dataset provided inside the spark-iceberg docker image
!ls -alh /home/iceberg/data/

total 574M
drwxr-xr-x 2 root root 4.0K Jul 25 15:10 .
drwxr-xr-x 1 root root 4.0K Jul 25 15:10 ..
-rw-r--r-- 1 root root 651K Jul 25 15:09 nyc_film_permits.json
-rw-r--r-- 1 root root  33M Jul 25 15:10 yellow_tripdata_2021-04.parquet
-rw-r--r-- 1 root root  37M Jul 25 15:10 yellow_tripdata_2021-05.parquet
-rw-r--r-- 1 root root  43M Jul 25 15:10 yellow_tripdata_2021-06.parquet
-rw-r--r-- 1 root root  42M Jul 25 15:10 yellow_tripdata_2021-07.parquet
-rw-r--r-- 1 root root  42M Jul 25 15:10 yellow_tripdata_2021-08.parquet
-rw-r--r-- 1 root root  44M Jul 25 15:10 yellow_tripdata_2021-09.parquet
-rw-r--r-- 1 root root  51M Jul 25 15:10 yellow_tripdata_2021-10.parquet
-rw-r--r-- 1 root root  51M Jul 25 15:10 yellow_tripdata_2021-11.parquet
-rw-r--r-- 1 root root  48M Jul 25 15:10 yellow_tripdata_2021-12.parquet
-rw-r--r-- 1 root root  37M Jul 25 15:10 yellow_tripdata_2022-01.parquet
-rw-r--r-- 1 root root  44M Jul 25 15:10 yellow_tripdata_2022-02.parquet
-rw-r--r-- 1 root root  54M Jul 25 1

In [9]:
parquet_df = spark.read.parquet("/home/iceberg/data/*.parquet")
parquet_df.printSchema()
parquet_df.show()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+----

In [15]:
parquet_df.count()

38908842

In [20]:
%%sql
CREATE DATABASE IF NOT EXISTS hms.nyc_notebook LOCATION 's3a://warehouse/nyc_notebook';

++
||
++
++

In [21]:
%%sql
CREATE TABLE IF NOT EXISTS hms.nyc_notebook.taxis
(
  vendor_id bigint,
  trip_id bigint,
  trip_distance float,
  fare_amount double,
  store_and_fwd_flag string
)
PARTITIONED BY (vendor_id);

++
||
++
++

In [11]:
from pyspark.sql.functions import col

columns = [
   col("VendorID").alias("vendor_id"),
   col("RatecodeID").alias("trip_id"), 
   "trip_distance", 
   "fare_amount", 
   "store_and_fwd_flag"
]
parquet_df.select(*columns).limit(1000000).writeTo("hms.nyc_notebook.taxis").overwritePartitions()

In [12]:
%%sql
select * from hms.nyc_notebook.taxis;

vendor_id,trip_id,trip_distance,fare_amount,store_and_fwd_flag
1,1,2.4000000953674316,10.0,N
1,1,2.200000047683716,10.5,N
1,1,14.0,43.5,N
1,1,5.5,17.5,N
1,1,1.2999999523162842,5.5,N
1,1,12.800000190734863,35.5,N
1,1,4.099999904632568,13.5,N
1,1,8.300000190734863,29.0,N
1,1,1.7999999523162842,8.0,N
1,1,12.199999809265137,33.5,N
